# PyOpenCL

You can program on GPUs using two main frameworks: (1) Cuda and (2) OpenCL. In a sense, these frameworks is a set of instructions to communate with the GPU and these frameworks have been implemented in many languages like Java and Python. Note, these are not the only frameworks but are the main ones.

Cuda is specifically for Nvidia GPUs, while OpenCL can be used for general GPUs. Due the speciality, Cuda tends to have a slight advantage over OpenCL. In fact, some games have special optimizations for Nvidia GPUs.

Cuda and OpenCL have slightly different syntaxes but are fundamentally the same. Note that Cupy is written in Cuda.

In [9]:
# For some reason we need this to install pyopencl on Colab
import locale
def getpreferredencoding(do_setlocale=True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [10]:
!pip install pyopencl

In [11]:
import numpy as np
import matplotlib.pyplot as plt

import pyopencl as cl

In [12]:
cl.get_platforms()

[<pyopencl.Platform 'Intel(R) OpenCL HD Graphics' at 0x1142ebec7c0>,
 <pyopencl.Platform 'Intel(R) OpenCL HD Graphics' at 0x1143027cd10>]

In [13]:
[platform.get_devices() for platform in cl.get_platforms()]

[[<pyopencl.Device 'Intel(R) Iris(R) Xe Graphics' on 'Intel(R) OpenCL HD Graphics' at 0x1142ea59b60>],
 [<pyopencl.Device 'Intel(R) Iris(R) Xe MAX Graphics' on 'Intel(R) OpenCL HD Graphics' at 0x1142ea58e60>]]

In [14]:
for platform in cl.get_platforms():
    for device in platform.get_devices():
        print(f"Name: {device.name}")
        print(f"Global Memory: {device.global_mem_size / 2**30} GB")
        print(f"Global Cache: {device.global_mem_cache_size / 2**10} KB")
        print(f"Local Memory: {device.local_mem_size / 2**10} KB")
        print(f"Compute Units: {device.max_compute_units}")
        print(f"Work Group Size: {device.max_work_group_size}")

Name: Intel(R) Iris(R) Xe Graphics
Global Memory: 6.29327392578125 GB
Global Cache: 1024.0 KB
Local Memory: 64.0 KB
Compute Units: 96
Work Group Size: 256
Name: Intel(R) Iris(R) Xe MAX Graphics
Global Memory: 3.1218719482421875 GB
Global Cache: 16384.0 KB
Local Memory: 64.0 KB
Compute Units: 96
Work Group Size: 256


# GPU Memory

Everything you do in Python, all the variables and datasets in Numpy, is stored in the CPU memory. However, the GPU memory is not shared with the CPU, so data must be sent from the CPU to the GPU. This typically is the major bottleneck of using GPUs, so you want to reduce the amount of data you want to send and receive from the CPU. More than that, the code that you want to run on the GPU must be 'programmed' on the GPU before it can be run.

Similarly, if you have multiple GPUs sending things to and from GPUs is also very expensive. So you can have instances where using two GPUs takes longer than using only one.

# Context and Queues

Context and queues are how you build and run things on the GPU.

A context is a namespace you want your programs and variables to live. Programs and variables in different contexts cannot see each other and cannot be added together. Using programs with different namespaces as your variables will also raise an error. You can think of the context as the memory shared and not with memory associated with another context.

Once the programs have been loaded and built in the context, we use queues to submit our jobs (known as events). Notably, submitting to the queue is performed asynchronously - once you submit your job to the queue, OpenCL will not wait for the job to finish, and the rest of your code will continue to run. This will only be an issue if the future code depends on the values of the previous code. In this case, you can wait for the job to finish before continuing on.

In [15]:
import pyopencl.array as cl_array
import numpy as np

import time

In [16]:
platform = cl.get_platforms()
devices = platform[0].get_devices()

In [17]:
context = cl.Context(devices)
queue = cl.CommandQueue(context)

In [18]:
queue2 = cl.CommandQueue(context)

In [19]:
x = np.random.rand(10_000).astype(np.float32)
y = np.random.rand(10_000).astype(np.float32)

# Send data to the gpu
x_gpu = cl_array.to_device(queue, x)
y_gpu = cl_array.to_device(queue, y)

[0.9965842  0.68383    0.44153908 ... 0.07874991 0.6782253  0.40646616]


In [25]:
# Perform addition on the gpu
out = x_gpu + y_gpu

print(x, y, out)

C:\Users\Will\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pyopencl\array.py:1222: DoubleDowncastWarning: The operation you requested would result in a double-precisision quantity according to numpy semantics. Since your device does not support double precision, a single-precision quantity is being returned.
  result = _get_broadcasted_binary_op_result(self, other, self.queue)


RuntimeError: clBuildProgram failed: BUILD_PROGRAM_FAILURE - clBuildProgram failed: BUILD_PROGRAM_FAILURE - clBuildProgram failed: BUILD_PROGRAM_FAILURE

Build on <pyopencl.Device 'Intel(R) Iris(R) Xe Graphics' on 'Intel(R) OpenCL HD Graphics' at 0x1142ea59b60>:

3:12:89: error: use of type 'double' requires cl_khr_fp64 extension to be enabled
        __kernel void axpbyz(__global float *z__base, long z__offset, float a, __global double *x__base, long x__offset, float b, __global float *y__base, long y__offset, long n)
                                                                                        ^
3:20:10: error: use of type 'double' requires cl_khr_fp64 extension to be enabled
__global double *x = (__global double *) ((__global char *) x__base + x__offset);
         ^
3:20:32: error: use of type 'double' requires cl_khr_fp64 extension to be enabled
__global double *x = (__global double *) ((__global char *) x__base + x__offset);
                               ^

(options: -I

# Elementwise Kernel

The element-wise kernel is the simplest way to start programming on a GPU but is very restrictive. If you have an operation that applies to each list element without requiring other elements, then the element-wise kernel is the best.

### Affine Operation

$$mx + b$$
for $m\in\mathbb{R}, x\in\mathbb{R}^n, b\in\mathbb{R}^n$

In [21]:
from pyopencl.elementwise import ElementwiseKernel

The arguments for `ElementwiseKernel` is
    
1. The context
2. The arguments to the function
3. The operation you want to apply to each element
3. The name of the function to be stored in the context

In [22]:
add_program = ElementwiseKernel(context, 
                                "double m, double *x, double *b, double *out",
                                "out[i] = m * x[i] + b[i]", 
                                "add")

In [23]:
# What happens if we change the type to np.float32
x = np.random.rand(10_000).astype(np.float64) - 0.5
b = np.random.rand(10_000).astype(np.float64)

x_gpu = cl_array.to_device(queue, x)
b_gpu = cl_array.to_device(queue, b)

In [24]:
out = cl_array.zeros_like(x_gpu)
add_program(np.float64(0.1), x_gpu, b_gpu, out)

RuntimeError: clBuildProgram failed: BUILD_PROGRAM_FAILURE - clBuildProgram failed: BUILD_PROGRAM_FAILURE - clBuildProgram failed: BUILD_PROGRAM_FAILURE

Build on <pyopencl.Device 'Intel(R) Iris(R) Xe Graphics' on 'Intel(R) OpenCL HD Graphics' at 0x1142ea59b60>:

2:12:27: error: use of type 'double' requires cl_khr_fp64 extension to be enabled
        __kernel void add(double m, __global double *x__base, long x__offset, __global double *b__base, long b__offset, __global double *out__base, long out__offset, long n)
                          ^
2:12:46: error: use of type 'double' requires cl_khr_fp64 extension to be enabled
        __kernel void add(double m, __global double *x__base, long x__offset, __global double *b__base, long b__offset, __global double *out__base, long out__offset, long n)
                                             ^
2:12:88: error: use of type 'double' requires cl_khr_fp64 extension to be enabled
        __kernel void add(double m, __global double *x__base, long 

You will notice that this returns an `Event`, meaning that this is happening asynchronously. So if you want to make sure the event is finished you need to do the following

In [ ]:
out = cl_array.zeros_like(x_gpu)
event = add_program(np.float64(0.1), x_gpu, b_gpu, out)
event.wait()

In [ ]:
np.testing.assert_almost_equal(out.get(), 0.1 * x + b)

### ReLu

In [ ]:
relu_program = ElementwiseKernel(context, 
                                 "double *x, double *out",
                                 "out[i] = x[i] > 0 ? x[i] : 0.0", 
                                 "relu")

### Sigmoid
The sigmoid function is defined as
$$f(x) = \frac{1}{1 + e^{-x}}.$$
But this function is famously is numerically unstable. As $x\to\infty,\ f(x)\to 1$ and as $x\to-\infty,\ f(x)\to 0$. This function is bounded, but not well behaved when $x \to -\infty$.

In [ ]:
def f(x):
    return 1 / (1 + np.exp(-x))

In [ ]:
x = 1e3
f(x)

1.0

In [ ]:
x = -1e3
f(x)

<ipython-input-26-a3ebedcd0b6b>:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


0.0

This is because of overflows, vs underflows

In [ ]:
def stable_f(x):
    if x > 0:
        return 1 / (1 + np.exp(-x))
    else:
        temp = np.exp(x)
        return  temp / (1 + temp)

In [ ]:
x = 1e3
stable_f(x)

1.0

In [ ]:
x = -1e3
stable_f(x)

0.0

In [ ]:
f(-100000)

<ipython-input-26-a3ebedcd0b6b>:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


0.0

In [ ]:
def stable_sigmoid(x):
    out = np.zeros_like(x)
    mask = x > 0

    out[mask] = 1 / (1 + np.exp(-x[mask]))

    temp = np.exp(x[~mask])
    out[~mask] = temp / (1 + temp)
    
    return out

In [ ]:
sigmoid_program = ElementwiseKernel(context,
                                    "double *x, double *out",
                                    "out[i] = SIGMOID(x[i])",
                                    "sigmoid",
                                    preamble='#define SIGMOID(x) x > 0 ? 1.0/(1.0 + exp(-x)) : exp(x) / (exp(x) + 1.0)'
                                    )

In [ ]:
x = (np.random.rand(10_000).astype(np.float64) - 0.5) * 1e4

x_gpu = cl_array.to_device(queue, x)

In [ ]:
out = cl_array.zeros_like(x_gpu)
sigmoid_program(x_gpu, out)

In [ ]:
np.testing.assert_almost_equal(stable_sigmoid(x), out.get())

# General Program

If our operation requires more involved access to other indices, we need to write our own kernel.

Before we get into that, a GPU is made up of thousands of workers, more specifically threads, who work in parallel. These workers have no structure to them, but we can 'organize' them up to 3-dimensions. These dimensions only help us organize the workers into easy layouts that may help certain programs.

These dimensions is accessed through `get_global_id`.

In [ ]:
add_c_code = """
__kernel void add(float m, __global float *a, __global float *b, __global float *out){
    int index = get_global_id(0);
    out[index] = m * a[index] + b[index];
}
"""

In [ ]:
program = cl.Program(context, add_c_code).build()

In [ ]:
a = np.random.rand(1000).astype(np.float32)
b = np.random.rand(1000).astype(np.float32)

a_gpu = cl_array.to_device(queue, a)
b_gpu = cl_array.to_device(queue, b)

In [ ]:
global_size = a_gpu.shape
local_size = None

out_gpu = cl_array.zeros_like(a_gpu)
program.add(queue, global_size, local_size, 
            np.float32(0.1), a_gpu.data, b_gpu.data, out_gpu.data)

In [ ]:
np.testing.assert_almost_equal(out_gpu.get(), 0.1 * a + b)

What happens if we change x into a matrix, rather than a vector

In [ ]:
x = np.random.rand(500, 500).astype(np.float32)
y = np.random.rand(500, 500).astype(np.float32)

x_gpu2 = cl_array.to_device(queue, x)
y_gpu2 = cl_array.to_device(queue, y)

In [ ]:
global_size = x_gpu2.shape
local_size = None

out_gpu = cl_array.zeros_like(x_gpu2)
program.add(queue, global_size, local_size, 
            np.float32(1), x_gpu2.data, y_gpu2.data, out_gpu.data)

In [ ]:
np.testing.assert_almost_equal(out_gpu.get(), x + y)

AssertionError: ignored

That didn't work. But why?

To fix it we need to change the `global_size`. The `global_size` defines how we distribute the workers. Before, we had the `global_size = (500, 500)`, so we split our workers into 2-dimensions with 500 in the 1st and 500 in the 2nd dimension. But our code only used `get_global_id(0)`, so eventhough all workers are in use, only it's first dimension is in use and the 2nd dimension becomes redundant.

We can fix this in two ways. 
1. First, we can use `get_global_id(1)` to use the 2nd dimension, or 
2. We can change `global_size = (500 * 500,)` and only use `get_global_id(0)`. Effectively this transforms the kernel into an element-wise kernel

In [ ]:
x = np.random.rand(500, 500).astype(np.float32)
y = np.random.rand(500, 500).astype(np.float32)

x_gpu2 = cl_array.to_device(queue2, x)
y_gpu2 = cl_array.to_device(queue2, y)

In [ ]:
global_size = (int(np.prod(x_gpu2.shape)),)
local_size = None

out_gpu = cl_array.zeros_like(x_gpu2)
program.add(queue, global_size, local_size, 
            np.float32(1), x_gpu2.data, y_gpu2.data, out_gpu.data)

In [ ]:
np.testing.assert_almost_equal(out_gpu.get(), x + y)

In [ ]:
add_c_code = """
__kernel void add(float m, __global float *a, __global float *b, __global float *out){
    int index = get_global_id(0);
    out[index] = m * a[index] + b[index];
}

__kernel void add_2d(float m, __global float *a, __global float *b, int width, __global float *out){
    int col = get_global_id(0);
    int row = get_global_id(1);

    int index = row * width + col;
    out[index] = m * a[index] + b[index];
}

__kernel void add_2d_v2(float m, __global float *a, __global float *b, int height, __global float *out){
    int row = get_global_id(0);
    int col = get_global_id(1);

    int index = row + col * height;
    out[index] = m * a[index] + b[index];
}
"""

In [ ]:
program = cl.Program(context, add_c_code).build()

In [ ]:
x = np.random.rand(5000, 500).astype(np.float32)
y = np.random.rand(5000, 500).astype(np.float32)

x_gpu2 = cl_array.to_device(queue, x)
y_gpu2 = cl_array.to_device(queue, y)

In [ ]:
local_size = None

height, width = x.shape

out_gpu = cl_array.zeros_like(x_gpu2)
event = program.add_2d(queue, x.shape[::-1], local_size, 
                       np.float32(1), x_gpu2.data, y_gpu2.data, np.int32(width), out_gpu.data)
event.wait()
np.testing.assert_almost_equal(out_gpu.get(), x + y)


out_gpu = cl_array.zeros_like(x_gpu2)
event = program.add_2d_v2(queue, x.shape, local_size, 
                          np.float32(1), x_gpu2.data, y_gpu2.data, np.int32(height), out_gpu.data)
event.wait()
np.testing.assert_almost_equal(out_gpu.get(), x + y)

In [ ]:
np.testing.assert_almost_equal(out_gpu.get(), x + y)

In [ ]:
%%timeit -r 7 -n 100
event = program.add_2d(queue, x.shape[::-1], local_size, 
                       np.float32(1), x_gpu2.data, y_gpu2.data, np.int32(width), out_gpu.data)
event.wait()

251 µs ± 21 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%%timeit -r 7 -n 100
event = program.add_2d_v2(queue, x.shape, local_size, 
                          np.float32(1), x_gpu2.data, y_gpu2.data, np.int32(height), out_gpu.data)
event.wait()

277 µs ± 36.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Transpose

As you may recall, the transpose of a matrix performs the operation $A_{i,j}\to A_{j,i}$. This can be implemented trivally using the 2D organization of the workers, as seen in `naive_transpose`.

You may have noticed the `__global` in the code, as well as the `global_size` and `local_size`. Like a CPU, GPUs have a global memory that is shared among all the workers but also a local memory that is shared with the local workers. As fast as global memory is, local memory is always going to be faster. 

### Aside - Row Major or Column Major
Let's say that we have a matrix
$$A = \begin{pmatrix}1 & 2 & 3\\ 4 & 5 & 6\\ 7 & 8 & 9\end{pmatrix}.$$

Numpy stores vectors and matrices as a contiguous memory block. Meaning that when you create a new matrix, Numpy will cut out a continuous chunk of memory and store all the numbers in that chunk of memory. It can store all the numbers by the row major (C-contiguous)
$$(1, 2, 3, 4, 5, 6, 7, 8, 9)$$
or by column major (Fortran-contiguous)
$$(1, 4, 7, 2, 5, 8, 3, 6, 9)$$

In [ ]:
import numpy as np

x = np.random.rand(3, 3)
x.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

In [ ]:
x.T.flags

  C_CONTIGUOUS : False
  F_CONTIGUOUS : True
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

Why is the transpose of a matrix Fortran-Contiguous not C-contiguous?


### Aside to the Aside - Non-contiguous Memory
It may or may not be hard to know what a non-contiguous memory block means. Non-contiguous memory follows a linked-list structure. 

https://levelup.gitconnected.com/array-vs-linked-list-data-structure-c5c0ff405f16


### Coalescing Memory
By default, OpenCl stores matrices a C-contiguous memory. So for a $[n\times m]$ matrix, the memory looks something like
$$(A_{0, 0}, A_{0, 1}, \cdots, A_{i,j}, A_{i, j + 1}, A_{i, j + 2}, A_{i, j + 3}, \cdots, A_{n, m-1}, A_{n, m})$$

Now the transpose requires access



### Coalescing Memory


https://www.cs.rochester.edu/~cding/Archive/MSP04Proceedings/p2_kawahito.pdf


In [ ]:
# https://github.com/inducer/pyopencl/blob/main/examples/transpose.py
# https://github.com/sschaetz/nvidia-opencl-examples/blob/master/OpenCL/src/oclTranspose/transpose.cl

In [ ]:
transpose_program = """
#define BLOCK_SIZE 16
#define A_BLOCK_STRIDE (BLOCK_SIZE * width)
#define A_T_BLOCK_STRIDE (BLOCK_SIZE * height)

__kernel void naive_transpose(__global float *a_t, __global float *a, int width, int height){
    unsigned int col = get_global_id(1);
    unsigned int row = get_global_id(0);

    a_t[col * height + row] = a[row * width + col];
}

__kernel void transpose(__global float *a_t, __global float *a, int width, int height, __local float *a_local){
    int global_col = get_global_id(0);
    int global_row = get_global_id(1);

    int local_col = get_local_id(0);
    int local_row = get_local_id(1);

    int local_index = local_row * BLOCK_SIZE + local_col;

    a_local[local_index] = a[global_row * width + global_col];

    barrier(CLK_LOCAL_MEM_FENCE);

    int group_col = get_group_id(0);
    int group_row = get_group_id(1);

    /* Transpose the blocks */
    global_row = group_col * BLOCK_SIZE + local_row;
    global_col = group_row * BLOCK_SIZE + local_col;

    a_t[global_row * height + global_col] = a_local[local_col * BLOCK_SIZE + local_row];
}
"""

In [ ]:
program = cl.Program(context, transpose_program).build()

In [ ]:
BLOCK_SIZE = 16

In [ ]:
x = np.random.rand(2 ** 13, 2 ** 13).astype(np.float32)
x_device = cl_array.to_device(queue, x)

x.shape

(8192, 8192)

In [ ]:
global_size = x_device.shape
local_size = (BLOCK_SIZE, BLOCK_SIZE)

width, height = x_device.shape

x_local = cl.LocalMemory(4 * BLOCK_SIZE * (BLOCK_SIZE + 1))
x_transpose = cl_array.zeros_like(x_device)

In [ ]:
event = program.transpose_v3(queue, global_size, local_size,
                               x_transpose.data, x_device.data, np.int32(width), np.int32(height), x_local)
event.wait()

In [ ]:
np.testing.assert_almost_equal(x_transpose.get(), x.T)

In [ ]:
%%timeit
event = program.naive_transpose(queue, global_size, local_size,
                                x_transpose.data, x_device.data, np.int32(width), np.int32(height))
event.wait()

4.03 ms ± 57.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%%timeit
event = program.transpose(queue, global_size, local_size,
                          x_transpose.data, x_device.data, np.int32(width), np.int32(height), x_local)
event.wait()

3.55 ms ± 6.66 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Cleaning It Up
It is a lot of code to write if we want to take the transpose of a matrix. To make it simpler, we will use classes

In [ ]:
class GPUTranspose:
    def __init__(self, context, queue):
        self.context = context
        self.queue = queue
        self.program = cl.Program(self.context, transpose_program).build()

    def transpose(self, x):
        """  x is assumed to be on the device """
        global_size = x.shape
        local_size = (16, 16)

        width, height = x.shape

        x_transpose = cl_array.zeros_like(x)
        a_local = cl.LocalMemory(4 * 16 * (16 + 1))
        
        self.program.transpose(self.queue, global_size, local_size,
                               x_transpose.data, x.data, np.int32(width), np.int32(height), a_local).wait()
        return x_transpose

    def naive_transpose(self, x):
        global_size = x.shape
        local_size = None

        x_transpose = cl_array.zeros_like(x)
        self.program.naive_transpose(queue, global_size, local_size,
                                     x_transpose.data, x.data, np.int32(width), np.int32(height)).wait()
        return x_transpose

In [ ]:
gpu_transpose = GPUTranspose(context, queue)

In [ ]:
x = np.random.rand(16 * 1000, 16 * 1000).astype(np.float32)
x_device = cl_array.to_device(queue, x)

In [ ]:
%%timeit
x_transpose = gpu_transpose.transpose(x_device)

27.2 ms ± 140 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
%%timeit
x_transpose = gpu_transpose.naive_transpose(x_device)

76.5 ms ± 660 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


# Question
1. Try and write the 2D heat equation solver using PyOpenCL
2. Try and write the sum of an array